# Gerando o Dataset De Entrada De Acidentes

### Importando bibliotecas

In [1]:
import pandas as pd

### Lendo a planilha de Acidentes Pre processada com os Locais de instalação

In [2]:
TabelaAcidentesPadronizada = pd.read_csv(
    "../dados/modificados/BD_Acidentes_Padronizado.csv", sep='|'
)
TabelaAcidentesPadronizada.shape

(994, 61)

## Filtrando para apenas Acidentes da Eletrosul

In [3]:
TabelaAcidentesPadronizada = TabelaAcidentesPadronizada[
    TabelaAcidentesPadronizada['Empresa'] == 'cgt eletrosul']
TabelaAcidentesPadronizada

,Empregado,Classificação,Nexo causal,Empresa,Fornecedor,Nº do Contrato,Segmento,Função,Sexo,Idade,...,Plano de ação,Cartão Alerta,Relatório de Investigação,year,month,day,hour,Data,Local de instalação,Distancia
0,proprio,tipico com afastamento,sim,cgt eletrosul,NaN,NaN,transmissao,tec manut prot apoio op,feminino,44.0,...,3.0,NaN,NaN,2020,2,11,9,2020-02-11 09:00:00,S-S-CHA,4.723856
1,terceiro,tipico com afastamento,sim,cgt eletrosul,fasttel engenharia s.a,NaN,transmissao,servente de obras,masculino,27.0,...,3.0,NaN,NaN,2020,2,1,14,2020-02-01 14:00:00,S-S-CMO,4.806254
2,proprio,tipico com afastamento,sim,cgt eletrosul,NaN,NaN,transmissao,tec manut equipamentos,masculino,45.0,...,3.0,NaN,NaN,2020,3,7,9,2020-03-07 09:00:00,S-S-CAN,3.591220
3,terceiro,tipico com afastamento,sim,cgt eletrosul,fasttel engenharia s.a,NaN,transmissao,servente de obras,masculino,25.0,...,3.0,NaN,NaN,2020,6,20,17,2020-06-20 17:00:00,S-S-CMO,6.796599
4,proprio,trajeto com afastamento,sim,cgt eletrosul,NaN,NaN,transmissao,tec eletricista lt,masculino,33.0,...,3.0,NaN,NaN,2020,9,24,12,2020-09-24 12:00:00,S-S-FAR,5.532086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,proprio,quase acidente,nao,cgt eletrosul,NaN,NaN,transmissao,enc. linhas de transmissao,masculino,53.0,...,NaN,NaN,NaN,2024,6,17,16,2024-06-17 16:00:00,S-E-BRU,8.136340
990,terceiro,tipico sem afastamento,sim,cgt eletrosul,rst engenharia e solucoes ltda,4500050325,csc,servente de limpeza,feminino,44.0,...,NaN,NaN,NaN,2024,6,21,10,2024-06-21 10:00:00,S-S-NSR,0.125647
991,proprio,quase acidente,nao,cgt eletrosul,NaN,NaN,transmissao,NaN,masculino,55.0,...,NaN,NaN,NaN,2024,6,19,14,2024-06-19 14:00:00,S-S-BIG,2.546664
992,proprio,quase acidente,nao,cgt eletrosul,NaN,NaN,geracao - hidraulica,tecnico de operacao e manutencao,masculino,41.0,...,NaN,NaN,NaN,2024,6,26,9,2024-06-26 09:00:00,S-S-ROQ,0.317754


### Contando a Quantidade de Acidentes em cada Local de instalação por Ano e Mês

In [4]:
def contagemAcidentes(TabelaAcidentes):
    # Supondo que cada linha na tabela é um acidente, podemos usar size() para contar
    TabelaAcidentes = (
        TabelaAcidentes.groupby(["Ano", "Mês", "Local de instalação"])
        .size()
        .reset_index(name="Quantidade de Acidentes")
    )
    return TabelaAcidentes

In [5]:
TabelaContagemAcidentes = TabelaAcidentesPadronizada.copy()
TabelaContagemAcidentes = contagemAcidentes(TabelaContagemAcidentes)
TabelaContagemAcidentes

,Ano,Mês,Local de instalação,Quantidade de Acidentes
0,2020,2,S-S-CHA,1
1,2020,2,S-S-CMO,1
2,2020,3,S-S-CAN,1
3,2020,6,S-S-CMO,1
4,2020,9,S-S-FAR,1
...,...,...,...,...
161,2024,6,S-S-BIG,1
162,2024,6,S-S-LIV2,2
163,2024,6,S-S-MRO2,1
164,2024,6,S-S-NSR,1


### Adicionando Colunas:
#### Quantidade de Acidentes de Alto Potencial e Binario Acidentes de Alto Potencial (1 se houver acidente de alto potencial, 0 caso contrário)

In [6]:
# Função para classificar de forma binaria se houve ou não acidentes de alto potencial
def tem_ou_nao(linha):
    if linha["Quantidade de Acidentes de Alto Potencial"] > 0:
        return 1
    else:
        return 0

In [7]:
# Acidentes de Alto Potencial são Acidentes com Potencial de 1 a 8
acidentesAltoPotencial = TabelaAcidentesPadronizada[TabelaAcidentesPadronizada["Potencial"] < 9]

acidentesPorMesAnoLocal = (
    TabelaAcidentesPadronizada.groupby(["Ano", "Mês", "Local de instalação"])
    .size()
    .reset_index(name="Quantidade De Acidentes")
)
acidentesAltoPotencialPorMesAnoLocal = (
    acidentesAltoPotencial.groupby(["Ano", "Mês", "Local de instalação"])
    .size()
    .reset_index(name="Quantidade De Acidentes")
)

# criar coluna de quantidade de acidentes de alto potencial na tabela acidentesPorMesAnoLocal
acidentesPorMesAnoLocal["Quantidade de Acidentes de Alto Potencial"] = (
    acidentesPorMesAnoLocal.apply(
        lambda x: (
            acidentesAltoPotencialPorMesAnoLocal[
                (acidentesAltoPotencialPorMesAnoLocal["Ano"] == x["Ano"])
                & (acidentesAltoPotencialPorMesAnoLocal["Mês"] == x["Mês"])
                & (
                    acidentesAltoPotencialPorMesAnoLocal["Local de instalação"]
                    == x["Local de instalação"]
                )
            ]["Quantidade De Acidentes"].values[0]
            if len(
                acidentesAltoPotencialPorMesAnoLocal[
                    (acidentesAltoPotencialPorMesAnoLocal["Ano"] == x["Ano"])
                    & (acidentesAltoPotencialPorMesAnoLocal["Mês"] == x["Mês"])
                    & (
                        acidentesAltoPotencialPorMesAnoLocal["Local de instalação"]
                        == x["Local de instalação"]
                    )
                ]["Quantidade De Acidentes"].values
            )
            > 0
            else 0
        ),
        axis=1,
    )
)

acidentesPorMesAnoLocal["Binario Acidentes de Alto Potencial"] = (
    acidentesPorMesAnoLocal.apply(tem_ou_nao, axis=1)
)

acidentesPorMesAnoLocal

,Ano,Mês,Local de instalação,Quantidade De Acidentes,Quantidade de Acidentes de Alto Potencial,Binario Acidentes de Alto Potencial
0,2020,2,S-S-CHA,1,0,0
1,2020,2,S-S-CMO,1,0,0
2,2020,3,S-S-CAN,1,0,0
3,2020,6,S-S-CMO,1,0,0
4,2020,9,S-S-FAR,1,0,0
...,...,...,...,...,...,...
161,2024,6,S-S-BIG,1,0,0
162,2024,6,S-S-LIV2,2,0,0
163,2024,6,S-S-MRO2,1,1,1
164,2024,6,S-S-NSR,1,0,0


### Transformando dataframe em um CSV

In [8]:
acidentesPorMesAnoLocal.to_csv(
    "../dados/modificados/acidentes_entrada.csv", index=False
)

## Validação do Dataset de Entrada de Acidentes

In [9]:
# Comparar quantidade de acidentes por ano e mes na tabela original com o novo dataset entrada acidentes

acidentesPorAnoMesOriginal = (
    TabelaAcidentesPadronizada.groupby(["Ano", "Mês"])
    .size()
    .reset_index(name="Quantidade de Acidentes Original")
)

acidentesPorAnoMes = (
    TabelaContagemAcidentes.groupby(["Ano", "Mês"])["Quantidade de Acidentes"]
    .sum()
    .reset_index()
)

# Left outer join para comprarar os datasets
resultado_join = pd.merge(
    acidentesPorAnoMesOriginal,
    acidentesPorAnoMes[["Ano", "Mês", "Quantidade de Acidentes"]],
    on=["Ano", "Mês"],
    how="left",
)

# Preencher NaN com 0 na coluna 'Quantidade de Acidentes'
resultado_join["Quantidade de Acidentes"] = resultado_join[
    "Quantidade de Acidentes"
].fillna(0)

# Para garantir que a coluna 'Quantidade de Acidentes' seja do tipo inteiro
resultado_join["Quantidade de Acidentes"] = resultado_join[
    "Quantidade de Acidentes"
].astype(int)

print(resultado_join)

     Ano  Mês  Quantidade de Acidentes Original  Quantidade de Acidentes
0   2020    2                                 2                        2
1   2020    3                                 1                        1
2   2020    6                                 1                        1
3   2020    9                                 1                        1
4   2021    1                                 5                        3
5   2021    2                                 2                        2
6   2021    3                                 2                        2
7   2021    6                                 1                        1
8   2021    7                                 2                        1
9   2021    8                                 2                        2
10  2021    9                                 3                        3
11  2021   10                                 8                        8
12  2021   11                                 1    

In [10]:
# somar o count total de acidentes de cada uma das tabelas
print(acidentesPorAnoMesOriginal["Quantidade de Acidentes Original"].sum())
print(acidentesPorAnoMes["Quantidade de Acidentes"].sum())

273
263


In [11]:
# usar tabela TabelaAcidentesPadronizada para contar quantidade de acidentes sem local de instalação
qtdAcidentesSemLocalInstalacao = TabelaAcidentesPadronizada["Local de instalação"].isna(
).sum()
print(qtdAcidentesSemLocalInstalacao)

10


## Conclusão da Validação:
### Diferenças entre os Datasets são causadas pela ausência do Local de Instação em 10 Linhas de Acidentes
Esses Acidentes estão há uma distancia maior do que 50 km (valor arbitrario) de qualquer Local de Instalação.